In [1]:
import pandas as pd


In [158]:
df = pd.read_csv('./../../scrapping_outputs/joined_data/cooking_oil/joined_result.csv')
product_type = 'cooking_oil'


In [119]:
def getRenameDict(related_columns,platform):
    returnDict = {}
    for i in related_columns:
        returnDict[i] = i.replace(platform+'_','')
    return returnDict
# 'flipkart_product'.replace('flipkart_','')

In [166]:
def saveProducts():
    for platform in ['flipkart','amazon']:
        related_columns = [col  for col in df.columns if platform in col]  
        filtered = df[related_columns]
        filtered.rename(columns=getRenameDict(related_columns,platform),inplace=True)
        filtered['source'] = platform
        filtered['name_hash'] = filtered[['product','source']].apply(lambda x: hash(tuple(x)), axis = 1)
        filtered = filtered.drop_duplicates(['name_hash'])
        filtered['prodcut_type'] = product_type
        filtered.to_csv(f'./products/{platform}_products.csv',)
saveProducts()

/home/tricon/.local/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/home/tricon/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [170]:
df = df[df.mached  == 'yes']
for platform in ['flipkart','amazon']:
    df[platform+'_soure'] = platform
    df[platform+'_id'] = df[[platform+'_product',platform+'_soure']].apply(lambda x: hash(tuple(x)), axis = 1)

joined_data = df[['flipkart_id','amazon_id','ratio']]
joined_data.to_csv('./intersection/intersection.csv')

In [171]:
import glob
path = './products/'

In [174]:
products = pd.concat([pd.read_csv(f) for f in glob.glob(path+'*.csv')]).drop_duplicates('name_hash')

In [191]:
flikart_data = joined_data.join(products.set_index('name_hash'),on ='flipkart_id' )
amazon_data  = joined_data.join(products.set_index('name_hash'),on ='amazon_id' )

In [253]:
l = []
for i in range(amazon_data.shape[0]):
     l.append({
        'name':amazon_data.iloc[i]['product'],
        'sources_info':[
            {
                'source':'Amazon',
                'cost':amazon_data.iloc[i]['price'],
                'link' : amazon_data.iloc[i]['product_link'],
            },
            {
                'source':'Flipkart',
                'cost':flikart_data.iloc[i]['price'],
                'link' : flikart_data.iloc[i]['product_link'],
            }
        ]})
        
        

In [254]:
l

[{'name': 'Gold Winner Refined Sunflower Oil, 5l',
  'sources_info': [{'source': 'Amazon',
    'cost': '₹709',
    'link': 'https://www.amazon.in//Gold-Winner-Refined-Sunflower-Oil/dp/B012AO394C/ref=sr_1_37_0o_fs?almBrandId=ctnow&fpw=alm&keywords=cooking+oil&qid=1638693875&sr=8-37'},
   {'source': 'Flipkart',
    'cost': '₹742',
    'link': 'https://www.flipkart.com/gold-winner-refined-sunflower-oil-can/p/itm6ad5d42f0389a?pid=EDOETZNK3BBCBXUZ&lid=LSTEDOETZNK3BBCBXUZ0KEE4Z&marketplace=GROCERY&q=cooking+oil&store=eat%2F18p&srno=s_1_14&otracker=search&fm=organic&iid=4748175e-5870-446d-81c2-072847a28b7e.EDOETZNK3BBCBXUZ.SEARCH&ppt=None&ppn=None&ssid=nhqhtwznv40000001638693857905&qH=6f7a90f8cb150708'}]},
 {'name': 'Freedom Refined Sunflower Oil, 1L',
  'sources_info': [{'source': 'Amazon',
    'cost': '₹144',
    'link': 'https://www.amazon.in//Freedom-Refined-Sunflower-Oil-1L/dp/B01BUZZ7H4/ref=sr_1_10_0o_fs?almBrandId=ctnow&fpw=alm&keywords=cooking+oil&qid=1638693875&sr=8-10'},
   {'source